# CSC 440 Programming Project
-- Student: Yangxin Fan
-- NetID: yfan24

# Data preprocessing

Read UCI Adult Census Dataset

In [1]:
import pandas as pd
import numpy as np
D = pd.read_csv("Adult.csv")
D

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48838,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
48839,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
48840,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


Let us analyze all categorical attributes

In [2]:
D = np.array(D)
D = D[:,[1,3,5,6,7,8,9,13,14]]
tran_set = D.tolist()
min_sup = 20000

Find all items in transactions

In [3]:
item_set = []
for tran in tran_set:
    item_set = list(set().union(item_set,tran))

# Algorithm I: Apriori [AS94b]
Find frequent itemsets using an iterative level-wise approach based on candidate generation

Find frequent 1-itemset

In [4]:
def find_frequent_1_itemsets(tran_set, item_set, min_sup):
    frequent_1_itemsets = []
    for item in item_set:
        count = 0
        for tran in tran_set:
            if item in tran: 
                count+=1
        if count >= min_sup:
            frequent_1_itemsets.append(item)
    return frequent_1_itemsets

import itertools
from itertools import combinations
L = find_frequent_1_itemsets(tran_set, item_set, min_sup)
L

['<=50K', 'White', 'United-States', 'Male', 'Private', 'Married-civ-spouse']

Function has_infrequent: Find whether a candidate itemset has a infrequent subset, if it has, 
then I would not be added (superset of an infrequent itemset must be infrequent)

In [5]:
import itertools
from itertools import combinations

def has_infrequent(c,L,k):
    subset = list(map(set, itertools.combinations(c, k-1)))
    for sub in subset:
        if all(sub != set(l) for l in L):
            return True
    return False

Function apriori_gen: generate candidates of frequent itemsets

In [6]:
def apriori_gen(L,k):
    C = []
    if k==2:
        C = list(itertools.combinations(L, 2))
    else:
        for p in L:
            for q in L:
                p.sort()
                q.sort()
                length = len(p)
                if all(p[i]==q[i] for i in range(length-1)) and p[length-1]<q[length-1]:
                    c = set(set().union(p,q))
                    if has_infrequent(list(c), L, k) is False:
                        C.append(list(c))
    return C

Generate a list of frequent itemsets

In [7]:
frequent_list = []
k = 2
#print (L)
frequent_list = frequent_list + L
while k < len(item_set) and len(L)>0:
    C = apriori_gen(L, k)
    #print(C)
    L = []
    #print(k)
    for candidate in C:
        count = 0
        for tran in tran_set:
            if set(candidate).issubset(set(tran)):
                count+=1
        if count >= min_sup and candidate not in L:
            L.append(list(candidate))
    #print (L)
    frequent_list = frequent_list + L
    k = k+1
frequent_list

['<=50K',
 'White',
 'United-States',
 'Male',
 'Private',
 'Married-civ-spouse',
 ['<=50K', 'White'],
 ['<=50K', 'United-States'],
 ['<=50K', 'Male'],
 ['<=50K', 'Private'],
 ['United-States', 'White'],
 ['Male', 'White'],
 ['Private', 'White'],
 ['Married-civ-spouse', 'White'],
 ['Male', 'United-States'],
 ['Private', 'United-States'],
 ['Married-civ-spouse', 'United-States'],
 ['Male', 'Private'],
 ['<=50K', 'United-States', 'White'],
 ['<=50K', 'Male', 'United-States'],
 ['<=50K', 'Private', 'White'],
 ['<=50K', 'Private', 'United-States'],
 ['Male', 'United-States', 'White'],
 ['Private', 'United-States', 'White'],
 ['<=50K', 'Private', 'United-States', 'White']]

# Algorithm II: FP-growth [HPY00]
Mine frequent itemsets using an FP-tree by pattern fragment growth

# Acquire a list of frequent items and their support counts and sort it by counts

First database scan

In [8]:
F = {}
for item in item_set:
    count = 0
    for tran in tran_set:
        if item in tran: 
            count+=1
    if count >= min_sup:
        F[item] = count
        
sorted_F = sorted(F.items(), key=lambda x: (-x[1],x[0]))
item_dict = {}
for pair in sorted_F:
    item_dict[pair[0]] = pair[1] 
item_dict

{'United-States': 43832,
 'White': 41762,
 '<=50K': 37155,
 'Private': 33906,
 'Male': 32650,
 'Married-civ-spouse': 22379}

# Construct FP-tree

Define the structure of the FP-Tree node 

In [9]:
class fpTreeNode:
    def __init__(self, name, freq, parent):
        self.name = name # contains the name of the item
        self.freq = freq # contains frequency of the item in the dataset
        self.parent = parent # contains the parent node
        self.child = OrderedDict() # contains all the children node information
        self.link = None # for linking to nodes with same name

Second scan D and create a sorted items list according to counts and a similary item dictionary that 
could link different nodes with same item together

In [10]:
sorted_F = sorted(F.items(), key=lambda x: (-x[1],x[0]))
sorted_keys = []
similar_item_dict = {}
for key in sorted_F:
        similar_item_dict[key[0]] = None # Initially all the values are 'None'
        sorted_keys.append(key[0])
similar_item_dict

{'United-States': None,
 'White': None,
 '<=50K': None,
 'Private': None,
 'Male': None,
 'Married-civ-spouse': None}

Go through each transaction and grow FP-tree

In [11]:
def fp_tree_create_and_update(init_node, trans, similar_item_dict):
    # if child is present, we increment its count
    if trans[0] in init_node.child:
        init_node.child[trans[0]].freq += 1
    # else, create a new node for the child and link it to its parent
    else:
        init_node.child[trans[0]] = fpTreeNode(trans[0], 1, init_node)
        # for every newly created node, the Similar-Item table is updated
        if similar_item_dict[trans[0]] == None:
            # for the 1st node, replace the 'None' value with the node
            similar_item_dict[trans[0]] = init_node.child[trans[0]]
        else:
            # traverse till the last similar node, and update the new node
            similar_item_table_update(similar_item_dict[trans[0]],\
                                      init_node.child[trans[0]])
    # recursively called for every item in a transaction
    if len(trans) > 1:
        fp_tree_create_and_update(init_node.child[trans[0]], trans[1::],\
                                  similar_item_dict)

In [12]:
def similar_item_table_update(similar_item, present_node): 
    while (similar_item.link != None):
        similar_item = similar_item.link
    similar_item.link = present_node

Use function similar_item_table_update, fp_tree_create_and_update to link nodes and update counts of nodes
and use class fpTreeNode to create new nodes 
Eventually, to build a FP-Tree

In [13]:
from collections import OrderedDict
root = fpTreeNode('Root',1,None)
for tr in tran_set:
    # delete all items whose frequency are below the minimum support
    tran = []
    for item in tr:
        if item in item_dict:
            tran.append(item)
    ordered_trans = []
    for item in sorted_keys:
        if item in tran:
            ordered_trans.append(item)
    if len(ordered_trans)!= 0:
        fp_tree_create_and_update(root, ordered_trans, similar_item_dict)

In [14]:
similar_item_dict

{'United-States': <__main__.fpTreeNode at 0x119019b20>,
 'White': <__main__.fpTreeNode at 0x10de26610>,
 '<=50K': <__main__.fpTreeNode at 0x1190196d0>,
 'Private': <__main__.fpTreeNode at 0x10de266d0>,
 'Male': <__main__.fpTreeNode at 0x10de26f70>,
 'Married-civ-spouse': <__main__.fpTreeNode at 0x10de26bb0>}

# Mining FP-tree

Create conditional pattern base

In [15]:
def create_cond_base(similar_item_dict, threshold, tran_set):
    final_cond_base = []
    # go through every key-value pair present in the Similar-Item Table
    for key,value in similar_item_dict.items():
        final_cond_base_key = []
        condition_base = []
        leaf_item_freq = OrderedDict()
        # within each key, traverse through every linked node value till the end
        while value != None:
            path = []
            leaf_node = value
            leaf_freq = value.freq
            # within each node, traverse till the parent node and append details
            while leaf_node.parent != None:
                leaf_details = [leaf_node.name, leaf_freq]
                path.append(leaf_details) # append the name and value
                leaf_node = leaf_node.parent # Go to the parent of that node
            # insert the whole path to condition_base
            condition_base.insert(0,path)
            # once the  particular node is finished, increment to value.link
            # then traverse for the next node with same name
            value = value.link
        # a frequent item-set dictionary is created for every leaf node
        for row in condition_base:
            for col in row:
                if col[0] not in leaf_item_freq:
                    leaf_item_freq[col[0]] = col[1]
                else:
                    leaf_item_freq[col[0]] += col[1]
        # items below threshold are removed before creating the conditional base
        leaf_item_freq = {k:v for k,v in leaf_item_freq.items() \
                          if v >= threshold}
        # for every transaction in the condition_base, the items are stored
        for row in condition_base:
            temp = []
            temp_tree = []
            for col in row:
                if col[0] in leaf_item_freq:
                    temp.append(col[0]) # stores only the name of the item
                    temp_tree.append(col) # stores both name and frequency
            # contains all the frequent items for a particular conditional leaf
            final_cond_base.append(temp) # used for question 2A
            final_cond_base_key.append(temp_tree) # used for question 2B
            
        cond_leaf = key
        cond_root = fpTreeNode('Root',1,None)
        # creates the conditional tree from the above conditonal pattern base
        for row in final_cond_base_key:
            conditional_fptree(cond_leaf,cond_root,row)
    # removes any duplicate rows having the same itemset
    unique_cond_base_set = set(map(tuple,final_cond_base))
    unique_cond_base_list =list(unique_cond_base_set)
    unique_cond_base = map(list,unique_cond_base_list)
    unique_cond_base_list.sort(key=lambda \
                               unique_cond_base_list:unique_cond_base_list[0])
    unique_cond_base = map(list,unique_cond_base_list)
    L = []
    for item_set in unique_cond_base_list:
        count = 0
        for tran in tran_set:
            if set(item_set).issubset(set(tran)):
                count+=1
        if count >= threshold:
            L.append(item_set)
    return L

   

In [16]:
def conditional_fptree(name,init_node,data):

    if data[0][0] == name:
        # skip the conditional tree if no extra frequent items are occuring
        if len(data)>1:
            conditional_fptree(name,init_node,data[1::])
    if data[0][0] != name:
        # if the item is present as a child, increment its count
        if data[0][0] in init_node.child:
            init_node.child[data[0][0]].freq += data[0][1]
        # else, create a new child node and update its frequency
        else:
            init_node.child[data[0][0]] = fpTreeNode(data[0][0],data[0][1],\
                           init_node)
        # continue to recursively create the conditional FP-Tree for each item
        if len(data) >1:
            conditional_fptree(name,init_node.child[data[0][0]],data[1::])

Create a set of frequent itemsets. 

In [17]:
minimal_freq_list = create_cond_base(similar_item_dict, min_sup, tran_set)
frequent_itemsets = set()
for item in minimal_freq_list:
    for i in range(1,len(item)+1):
        for set_freq in itertools.combinations(item, i):
            frequent_itemsets.add(set_freq)
frequent_itemsets

{('<=50K',),
 ('<=50K', 'United-States'),
 ('<=50K', 'White'),
 ('<=50K', 'White', 'United-States'),
 ('Male',),
 ('Male', '<=50K'),
 ('Male', '<=50K', 'United-States'),
 ('Male', 'Private'),
 ('Male', 'United-States'),
 ('Male', 'White'),
 ('Male', 'White', 'United-States'),
 ('Married-civ-spouse',),
 ('Married-civ-spouse', 'United-States'),
 ('Married-civ-spouse', 'White'),
 ('Private',),
 ('Private', '<=50K'),
 ('Private', '<=50K', 'United-States'),
 ('Private', '<=50K', 'White'),
 ('Private', '<=50K', 'White', 'United-States'),
 ('Private', 'United-States'),
 ('Private', 'White'),
 ('Private', 'White', 'United-States'),
 ('United-States',),
 ('White',),
 ('White', 'United-States')}

# Algorithm III: Improving the Efficiency of Apriori Using Transaction Reduction

In [18]:
import pandas as pd
import numpy as np
D = np.array(pd.read_csv("Adult.csv"))
min_sup = 20000
D = D[:,[1,3,5,6,7,8,9,13,14]]
tran_set = D.tolist()

In [19]:
item_set = []
for tran in tran_set:
    item_set = list(set().union(item_set,tran))

In [20]:
def find_frequent_1_itemsets(tran_set, item_set, min_sup):
    frequent_1_itemsets = []
    for item in item_set:
        count = 0
        for tran in tran_set:
            if item in tran: 
                count+=1
        if count >= min_sup:
            frequent_1_itemsets.append(item)
    return frequent_1_itemsets

import itertools
from itertools import combinations
L = find_frequent_1_itemsets(tran_set, item_set, min_sup)

In [21]:
import itertools
from itertools import combinations

# pass p, q to has_infrequent, in subset we do not consider these 2 subsets (already frequent)
def has_infrequent(c,L,k,p,q):
    subset = list(map(set, itertools.combinations(c, k-1)))
    
    # --- added part start
    subset.remove(set(p))
    subset.remove(set(q))
    # --- added part end
    
    for sub in subset:
        if all(sub != set(l) for l in L):
            return True
    return False

In [22]:
def apriori_gen(L,k):
    C = []
    if k==2:
        C = list(itertools.combinations(L, 2))
    else:
        for p in L:
            for q in L:
                p.sort()
                q.sort()
                length = len(p)
                if all(p[i]==q[i] for i in range(length-1)) and p[length-1]<q[length-1]:
                    c = set(set().union(p,q))
                    if has_infrequent(list(c), L, k, p, q) is False:
                        C.append(list(c))
    return C

Remove a transaction if it does not contain any frequent k-itemsets so we do not need 
to scan such a transaction when finding the count of k+1 candidate itemsets

In [23]:
frequent_list = []
k = 2
frequent_list = frequent_list + L
while k < len(item_set) and len(L)>0:
    
    # --- added part start
    for tran in tran_set:
        count = 0
        for freq_item in L:
            if k == 2:
                if freq_item in tran:
                    count+=1
            else:
                if set(freq_item).issubset(set(tran)):
                    count+=1   
        if count==0:
            tran_set.remove(tran)
    # --- added part end
            
    C = apriori_gen(L, k)
    L = []
    
    for candidate in C:
        count = 0
        # --- added part start
        for tran in tran_set:
            if len(tran)<len(candidate):
                tran_set.remove(tran)
        # --- added part end
            
        for tran in tran_set:
            if set(candidate).issubset(set(tran)):
                count+=1
        if count >= min_sup and candidate not in L:
            L.append(list(candidate))
    frequent_list = frequent_list + L
    k = k+1
frequent_list

['<=50K',
 'White',
 'United-States',
 'Male',
 'Private',
 'Married-civ-spouse',
 ['<=50K', 'White'],
 ['<=50K', 'United-States'],
 ['<=50K', 'Male'],
 ['<=50K', 'Private'],
 ['United-States', 'White'],
 ['Male', 'White'],
 ['Private', 'White'],
 ['Married-civ-spouse', 'White'],
 ['Male', 'United-States'],
 ['Private', 'United-States'],
 ['Married-civ-spouse', 'United-States'],
 ['Male', 'Private'],
 ['<=50K', 'United-States', 'White'],
 ['<=50K', 'Male', 'United-States'],
 ['<=50K', 'Private', 'White'],
 ['<=50K', 'Private', 'United-States'],
 ['Male', 'United-States', 'White'],
 ['Private', 'United-States', 'White'],
 ['<=50K', 'Private', 'United-States', 'White']]

# Summary of improved Apriori with three improvements

1. Less scan of transactions in database: removing a transaction if it does not contain any frequent k-itemsets 
so we do not need to scan such a transaction when finding the count of k+1 candidate itemsets
2. When pruning candidate itemsets, we do not need to check two itemsets where a particular itemset generated
from since they are already frequent itemsets
3. Delete transactions where their length less than the length of candidate itemset we want to check for frequency

Reference:
https://towardsdatascience.com/understand-and-build-fp-growth-algorithm-in-python-d8b989bab342